## Machine Translation Project

In this notebook, I built several deep neural networks that functions as part of an end-to-end machine translation pipeline. Completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - Converts text to sequence of integers.
- **Models** Models which accepts a sequence of integers as input and returns a probability distribution over possible translations. 
- **Prediction** Run the model on English text.

#### Models explored
- Simple RNN
- RNN with word embeddings
- Bidirectional RNN
- Bidirectional RNN with embeddings
- **Auto Encoder-Decoder**

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

### Import necessary libraries & Verify access to the GPU

In [3]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13856438438791511884
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357236736
locality {
  bus_id: 1
}
incarnation: 14211666978726769005
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate Machine Translation (MT) pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  Since the purpose of this exercise is to just explore the MT models we'll be using a dataset that contains a small vocabulary. 
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Below, English and French data is loaded from these files.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  Priniting the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  But the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


## Preprocess
For this project, we won't be using text data as input to model. Instead, we'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

### Tokenize
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

We will be using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function to tokenize `english_sentences` and `french_sentences` in the cell below.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    """
    Tokenizer class allows to vectorize a text corpus, by turning each text into either a sequence of integers 
    (each integer being the index of a token in a dictionary) 
    or into a vector where the coefficient for each token could be binary, based on word count, based on tf-idf...
    
    .fit_on_texts(corpus) to fit the model on corpus
    .texts_to_sequences(x) converts list of txt to list of lists containing tokens
    """
    x_tk = Tokenizer() # keras model
    x_tk.fit_on_texts(x) # fit model on corpus

    return x_tk.texts_to_sequences(x), x_tk # read more about this

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Below, we are making sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length is None:
        length = max([len(sen) for sen in x])
    
    return pad_sequences(x, maxlen=length, padding='post') # keras padding function
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
It tokenizes and pads the sentences.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
Initially, I trained four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with these four simple architectures, I constructed a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gap between the logits from the neural network to the French translation.  We'll be using this function to better understand the output of the neural network to debug and validate.

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, we train a RNN that translates English to French.

In [11]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    
    # TODO: Build the layers
    inputs = Input(shape=input_shape[1:])
    rnn = GRU(64, return_sequences=True)(inputs)
    x = Dense(128, activation='relu')(rnn)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(x)
    
    model = Model(inputs=inputs, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 9s 82us/step - loss: 3.2104 - acc: 0.4389 - val_loss: nan - val_acc: 0.4743
Epoch 2/10
110288/110288 [==============================] - 7s 64us/step - loss: 2.2029 - acc: 0.5013 - val_loss: nan - val_acc: 0.5186
Epoch 3/10
110288/110288 [==============================] - 7s 64us/step - loss: 1.8502 - acc: 0.5447 - val_loss: nan - val_acc: 0.5693
Epoch 4/10
110288/110288 [==============================] - 7s 64us/step - loss: 1.6132 - acc: 0.5819 - val_loss: nan - val_acc: 0.5904
Epoch 5/10
110288/110288 [==============================] - 7s 64us/step - loss: 1.4780 - acc: 0.6019 - val_loss: nan - val_acc: 0.6161
Epoch 6/10
110288/110288 [==============================] - 7s 64us/step - loss: 1.3912 - acc: 0.6149 - val_loss: nan - val_acc: 0.6199
Epoch 7/10
110288/110288 [==============================] - 7s 64us/step - loss: 1.3303 - acc: 0.6285 - val_loss: nan - val_acc: 0.63

### Model 2: Embedding
![RNN](images/embedding.png)
There's a better representation of a word ids. This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, I trained a RNN model using embedding.

In [12]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    
    # TODO: Build the layers
    #print(input_shape)
    inputs = Input(shape=input_shape[1:])
    embeds = Embedding(output_dim=51, input_dim=english_vocab_size, input_length=input_shape[1])(inputs)
    rnn = GRU(64, return_sequences=True)(embeds)
    x = Dense(128, activation='relu')(rnn)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(x)
    
    model = Model(inputs=inputs, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_embed_model(embed_model)

# no need to reshape, we need to send it as a two dimensional array
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
print(tmp_x.shape)

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(137861, 21)
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 8s 76us/step - loss: 3.5052 - acc: 0.3979 - val_loss: nan - val_acc: 0.4296
Epoch 2/10
110288/110288 [==============================] - 8s 72us/step - loss: 2.2369 - acc: 0.4986 - val_loss: nan - val_acc: 0.5610
Epoch 3/10
110288/110288 [==============================] - 8s 71us/step - loss: 1.4434 - acc: 0.6353 - val_loss: nan - val_acc: 0.7017
Epoch 4/10
110288/110288 [==============================] - 8s 71us/step - loss: 1.0308 - acc: 0.7344 - val_loss: nan - val_acc: 0.7570
Epoch 5/10
110288/110288 [==============================] - 8s 73us/step - loss: 0.8535 - acc: 0.7681 - val_loss: nan - val_acc: 0.7805
Epoch 6/10
110288/110288 [==============================] - 8s 73us/step - loss: 0.7443 - acc: 0.7891 - val_loss: nan - val_acc: 0.7970
Epoch 7/10
110288/110288 [==============================] - 8s 72us/step - loss: 0.6638 - acc: 0.8072 - val_loss: nan - 

### Model 3: Bidirectional RNNs
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [13]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    
    inputs = Input(shape=input_shape[1:])
    #embeds = Embedding(output_dim=51, input_dim=english_vocab_size, input_length=input_shape[1])(inputs)
    rnn = Bidirectional(GRU(64, return_sequences=True))(inputs)
    x = Dense(128, activation='relu')(rnn)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(x)
    model = Model(inputs=inputs, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_bd_model(bd_model)


# need to reshape, we need to send it as a 3 dimensional array
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 11s 100us/step - loss: 2.9867 - acc: 0.4820 - val_loss: nan - val_acc: 0.5616
Epoch 2/10
110288/110288 [==============================] - 10s 94us/step - loss: 1.7264 - acc: 0.5825 - val_loss: nan - val_acc: 0.5963
Epoch 3/10
110288/110288 [==============================] - 11s 97us/step - loss: 1.4768 - acc: 0.6103 - val_loss: nan - val_acc: 0.6233
Epoch 4/10
110288/110288 [==============================] - 11s 96us/step - loss: 1.3516 - acc: 0.6315 - val_loss: nan - val_acc: 0.6390
Epoch 5/10
110288/110288 [==============================] - 11s 96us/step - loss: 1.2612 - acc: 0.6453 - val_loss: nan - val_acc: 0.6509
Epoch 6/10
110288/110288 [==============================] - 11s 97us/step - loss: 1.1990 - acc: 0.6564 - val_loss: nan - val_acc: 0.6604
Epoch 7/10
110288/110288 [==============================] - 11s 96us/step - loss: 1.1528 - acc: 0.6638 - val_loss: nan - val_a

### Model 4: Encoder-Decoder 
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

In [14]:
def enc_dec_model(enc_input_shape, dec_input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    
    # TODO: Build the layers
    #print(input_shape)
    enc_inputs = Input(shape=enc_input_shape[1:])
    enc_embeds = Embedding(output_dim=51, input_dim=english_vocab_size, input_length=enc_input_shape[1])(enc_inputs)
    enc_outputs, state_h = GRU(64, return_sequences=True, return_state=True)(enc_embeds)
    
    dec_inputs = Input(shape=dec_input_shape[1:])
    # input_dim, french_vocab_size + start tag
    dec_embeds = Embedding(output_dim=51, input_dim=french_vocab_size+1, input_length=dec_input_shape[1])(dec_inputs)
    dec_outputs = GRU(64, return_sequences=True)(dec_embeds, initial_state=state_h)
    
    x = Dense(128, activation='relu')(dec_outputs)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(x)
    
    model = Model(inputs=[enc_inputs, dec_inputs], outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# encoder input
# no need to reshape, we need to send it as a two dimensional array
enc_x = pad(preproc_english_sentences, max_french_sequence_length)
print(enc_x.shape)

# decoder input
# no need to reshape, we need to send it as a two dimensional array
# covert 3d to 2d array
preproc_french_sentences_2d = np.squeeze(preproc_french_sentences, axis=2)
# add start tag to begining of every french sentences
strt_tag_val = 345
preproc_french_sentences_2d_strt = np.insert(preproc_french_sentences_2d, 0, strt_tag_val, axis=1)
dec_x = pad(preproc_french_sentences_2d_strt, max_french_sequence_length+1)
print(dec_x.shape)

# add 0 at the last position of array
dec_y = np.insert(preproc_french_sentences, -1, 0, axis=1)

# Train the neural network
encdec_rnn_model = enc_dec_model(
    enc_x.shape, dec_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
encdec_rnn_model.fit([enc_x, dec_x], dec_y, batch_size=1024, epochs=10, validation_split=0.2)

(137861, 21)
(137861, 22)
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 13s 119us/step - loss: 3.3393 - acc: 0.4507 - val_loss: nan - val_acc: 0.4863
Epoch 2/10
110288/110288 [==============================] - 12s 113us/step - loss: 1.8632 - acc: 0.5513 - val_loss: nan - val_acc: 0.6454
Epoch 3/10
110288/110288 [==============================] - 13s 114us/step - loss: 1.0723 - acc: 0.6975 - val_loss: nan - val_acc: 0.7224
Epoch 4/10
110288/110288 [==============================] - 12s 113us/step - loss: 0.8265 - acc: 0.7387 - val_loss: nan - val_acc: 0.7483
Epoch 5/10
110288/110288 [==============================] - 13s 113us/step - loss: 0.7306 - acc: 0.7559 - val_loss: nan - val_acc: 0.7601
Epoch 6/10
110288/110288 [==============================] - 13s 114us/step - loss: 0.6766 - acc: 0.7673 - val_loss: nan - val_acc: 0.7711
Epoch 7/10
110288/110288 [==============================] - 13s 114us/step - loss: 0.6408 - acc

In [ ]:
# create a zero matrix just like decoder input 
dec_x_pred = np.zeros_like(dec_x)
# add starting start tag to first element of each sentences
dec_x_pred[:, 0] = 345
for i in range(1,22):
    print(i)
    # get prediction for 1st word and then add it to dec_x_pred matrix
    dec_x_pred[:, i] = np.argmax(encdec_rnn_model.predict([enc_x, dec_x_pred]), axis=2)[:, i-1]

In [ ]:
def enc_dec_to_text(enc_dec_output, tokenizer):
    """
    Turn encoder decoder final output into text using the tokenizer
    enc_dec_out: output of encoder and decoder
    tokenizer: language tokenizer
    """
    
    print('converting integers to words......')
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    index_to_words[345] = '<start>' 

    return np.vectorize(index_to_words.get)(enc_dec_output)

### Model 5: Custom
A bidirectional rnn with embedding vector model.

In [52]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    
    inputs = Input(shape=input_shape[1:])
    embeds = Embedding(output_dim=51, input_dim=english_vocab_size, input_length=input_shape[1])(inputs)
    
    #embeds = Embedding(output_dim=51, input_dim=english_vocab_size, input_length=input_shape[1])(inputs)
    rnn = Bidirectional(GRU(64, return_sequences=True))(embeds)
    x = Dense(128, activation='relu')(rnn)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(x)
    model = Model(inputs=inputs, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

#tests.test_model_final(model_final)

# no need to reshape, we need to send it as a two dimensional array
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
print(tmp_x.shape)

# Train the neural network
embed_rnn_model = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=60, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))
print('Final Model Loaded')
# TODO: Train the final model

(137861, 21)
Train on 110288 samples, validate on 27573 samples
Epoch 1/60
110288/110288 [==============================] - 14s 123us/step - loss: 3.2989 - acc: 0.4431 - val_loss: nan - val_acc: 0.4953
Epoch 2/60
110288/110288 [==============================] - 12s 111us/step - loss: 2.0400 - acc: 0.5408 - val_loss: nan - val_acc: 0.6182
Epoch 3/60
110288/110288 [==============================] - 12s 111us/step - loss: 1.2738 - acc: 0.6789 - val_loss: nan - val_acc: 0.7362
Epoch 4/60
110288/110288 [==============================] - 12s 111us/step - loss: 0.9416 - acc: 0.7592 - val_loss: nan - val_acc: 0.7790
Epoch 5/60
110288/110288 [==============================] - 12s 111us/step - loss: 0.7710 - acc: 0.7918 - val_loss: nan - val_acc: 0.8035
Epoch 6/60
110288/110288 [==============================] - 12s 111us/step - loss: 0.6626 - acc: 0.8135 - val_loss: nan - val_acc: 0.8231
Epoch 7/60
110288/110288 [==============================] - 12s 111us/step - loss: 0.5838 - acc: 0.8294 - va

## Prediction

In [53]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = embed_rnn_model

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [english_tokenizer.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=preproc_french_sentences.shape[1], padding='post')
    sentences = np.array([list(sentence[0]), list(pad_sequences(x, maxlen=max_french_sequence_length, padding='post')[0])])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Sample 1:
il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
new jersey est parfois calme au l' automne et il est neigeux en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Note

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function can be used to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set.